In [17]:
import pandas as pd

%run DataSource.ipynb
%run Metric.ipynb

report_columns = ['ma_short','ma_mid','total_profit',
                  'win_rate','loss_times','max_retrace',
                  'deal_count','win_count',
                  'period_score','retrace_score']

Buy_type='Buy'
Sell_type='Sell'
deal_open='open'
deal_closed='done'

class Account:
    init_money = 0
    start_date = ''
    account_money = 0
    account_shares = 0
    buy_price = []
    buy_date = []
    floating_profits = []
    operation_detail= pd.DataFrame(columns=['date','profit_rate','account_money'])
    daily_log=True
    
    def __init__(self,init_money,start_date,end_date,short,mid):
        self.init_money = init_money
        self.account_money = init_money
        
        self.start_date = start_date
        self.end_date = end_date
        
        self.short = short
        self.mid = mid
        
        self.account_shares = 0
        self.buy_symbol = []
        self.buy_price = []
        self.buy_date = []
        self.space = []
        self.floating_profits = []
        
        self.i = 0
        self.operation_detail = pd.DataFrame(columns=['date','profit_rate','account_money'])
        self.operation_detail.loc[self.i] = [start_date,0,self.init_money]
        self.i +=1
        
        self.daily_log=True
        
        self.deal_df = pd.DataFrame(columns = ['symbol', 'date', 'price', 'shares', 
                                               'type', 'space','status'])
        self.position = {}
        self.metric_d_k = {}
        self.asset = pd.DataFrame(columns = ['date','assets'])
        
    def get_init_money(self):
        return self.init_money
    
    def get_all_cash(self):
        return self.account_money
    
    def buy_stock(self,symbol,date,price,buy_shares):
        space_value = 100*(price*buy_shares)/self.account_money
        self.deal_df.loc[self.deal_df.shape[0] + 1] = [symbol,date,price,buy_shares,Buy_type, space_value,'open']
        
        if(symbol in self.position.keys()):
            print('Warning, ' +symbol+' 加仓 ' + buy_shares +' shares')
            self.position[symbol] = self.position[symbol] + buy_shares 
        else:
            print(date +' 开仓 ' + symbol +' ' +str(buy_shares) +' shares')
            self.position[symbol] = buy_shares
         
        self.account_money -= buy_shares*price
        self.metric_d_k[symbol]=Metric(symbol, level_day, self.short, self.mid)
    
    def sell_stock(self,symbol,date,price,sell_shares):
        if(symbol in self.position.keys()):
            self.position[symbol] = self.position[symbol] - sell_shares 
        else:
            print('Warning!!, dirty data ' +symbol+' '+ date+ ' has no shares')
        
        self.deal_df.loc[self.deal_df.shape[0] + 1] = [symbol,date,price,sell_shares,Sell_type, 0, deal_open]
        
        if(self.position[symbol]<=0):
            print(date +' 清仓 ' + symbol +' ' +str(sell_shares) +' shares')
            del(self.position[symbol])
            self.deal_df.loc[(self.deal_df['symbol']==symbol) & (self.deal_df['status']==deal_open),'status']=deal_closed
    
        self.account_money += sell_shares*price
        
    def get_shares(self,symbol):
        return self.position[symbol]

    def can_open_new_stock(self):
        for key in self.position:
            if(self.position[key]>0):
                return False
            
        return True
        
    def has_shares(self):
        for item in self.position.values():
            if(item>0):
                return True

        return False
    
    
    def get_latest_deal(self,symbol):
        deals = self.deal_df[(self.deal_df['symbol']==symbol) & 
                                   (self.deal_df['type']==Buy_type)]
        
        return deals[deals.shape[0]-1:]
        
    
    def get_last_time_buy_price(self,symbol):
        latest_deal = self.get_latest_deal(symbol)
        
        return latest_deal['price'].values[0]
    
    def get_last_time_buy_date(self,symbol):
        latest_deal = self.get_latest_deal(symbol)
        
        return latest_deal['date'].values[0]
        

    def buy_and_sell(start_date,end_date,report_summary, short,mid):
        print('start go through MA(' + str(short) + ',' + str(mid) + ')')
        all_dates = data['date'][mid:]

        account = Account(initial_money,start_date,short,mid)

        start_year_s, start_mon_s, start_day_s = start_date.split('-')
        start_datetime = datetime(int(start_year_s), int(start_mon_s), int(start_day_s))

        end_year_s, end_mon_s, end_day_s = end_date.split('-')
        end_datetime = datetime(int(end_year_s), int(end_mon_s), int(end_day_s))

        for date in all_dates:
            if(not(out_of_range(date,start_datetime,end_datetime))):
                price = data[data['date']==date]['close'].values[0]
                account.daily_check(date,price)

        report_data = account.report()
        report_summary.loc[report_summary.shape[0]+1] = report_data

        return report_summary
    
    def get_all_position_symbols(self):
        return self.position.keys()
    
    def get_asset_df(self):
        return self.asset
    
    
    def get_today_asset(self,cur_date):
        all_symbols = self.position.keys()
        
        today_asset = self.account_money
        if(len(all_symbols)>0):
            stock_asset = 0
            for symbol in all_symbols:
                d_k_metric = self.metric_d_k[symbol]
                cur_price =  d_k_metric.get_cur_price(cur_date)
                stock_asset += cur_price*self.position[symbol]
            
            today_asset = self.account_money + stock_asset
        
        return today_asset
        
    
    def daily_audit(self,cur_date):
        today_asset = self.get_today_asset(cur_date)
        
        self.asset.loc[self.asset.shape[0]+1] = [cur_date,today_asset]
        

        